In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('/home/chenxi/data/2023_LoL_esports_match_data_from_OraclesElixir.csv')

/home/chenxi/.conda/envs/myen/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
cleaned_data = data[data["position"] == "team"]
cleaned_data[["firstblood", "firstdragon", "firsttower"]] = cleaned_data[["firstblood", "firstdragon", "firsttower"]].fillna(0)
cleaned_data["firstblood"]= cleaned_data["firstblood"].astype(int).astype(str)
cleaned_data["firstdragon"] = cleaned_data["firstdragon"].astype(int).astype(str)

cleaned_data["firsttower"] = cleaned_data["firsttower"].astype(int).astype(str)

/home/chenxi/.conda/envs/myen/lib/python3.10/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/tmp/ipykernel_3227709/4225977938.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data["firstblood"]= cleaned_data["firstblood"].astype(int).astype(str)
/tmp/ipykernel_3227709/4225977938.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import QuantileTransformer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [5]:
cleaned_data['kda_15'] = (cleaned_data['killsat15'] + cleaned_data['assistsat15']) / (1+cleaned_data['deathsat15'])
cleaned_data['gold_xp_ratio_at15'] = cleaned_data['goldat15'] / (cleaned_data['xpat15'] + 1)  # Avoid division by zero


/tmp/ipykernel_3227709/3788205313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['kda_15'] = (cleaned_data['killsat15'] + cleaned_data['assistsat15']) / (1+cleaned_data['deathsat15'])
/tmp/ipykernel_3227709/3788205313.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['gold_xp_ratio_at15'] = cleaned_data['goldat15'] / (cleaned_data['xpat15'] + 1)  # Avoid division by zero


In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score



features = ["goldat10", "firstblood", "firstdragon", "firsttower", "xpat15", "goldat15", 
            "firstmidtower", "kda_15", "league"]
cleaned_data[features] = cleaned_data[features].fillna(0)
X = cleaned_data[features]
y = cleaned_data['result']
numeric_features = ["goldat10", "xpat15", "goldat15", "kda_15"]
categorical_features = ["firstblood", "firstdragon", "firsttower", "firstmidtower", "league"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
     ("scaker",  QuantileTransformer(n_quantiles=100))
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestClassifier(random_state=42))
])


param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(rf_pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_rf = grid_search.best_estimator_
predictions = best_rf.predict(X_test)

/home/chenxi/.conda/envs/myen/lib/python3.10/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [7]:
accuracy_score(y_test, predictions)

0.7417202763878962

In [8]:
f1_score(y_test, predictions)

0.73954829408938

In [9]:
from joblib import dump
filename = '/home/chenxi/data/DSC148/final/my_model.joblib'

# Save the model to a file
dump(grid_search, filename)

print(f"Model saved to {filename}")

Model saved to /home/chenxi/data/DSC148/final/my_model.joblib


In [10]:
import pickle

# Assuming `best_rf` is your trained model
model = grid_search

# Save the model to disk
with open('model_filename.pkl', 'wb') as file:
    pickle.dump(model, file)

